In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go

#Global var
init_cap = 2000
price_limit = 5 # don't open position if price exceeded price_limit
risk_free_rate = 5.0

#directory path
dir_path =r'D:\OneDrive\stock\Leleka\BuyBeforeEarnings\inputForPython'
csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]

if not csv_files:
    print('No CSV file in directory')



### Working with stock list filter dataframe

In [2]:

# add stock list to filter data
df_stock = pd.read_excel('InputStocks.xlsx')
# Using the str.cat() method without the underscore separator
df_stock['Y_Symbol'] = df_stock['Year'].astype(str).str.cat(df_stock['Symbol'], sep='')
df_stock['Y_Symbol'] = df_stock['Y_Symbol'].str.replace(' ', '')

In [122]:

#df_stock.to_excel('df_stock_result.xlsx')


### Working with CML download files

In [3]:
# Add all files in folder in one file
dataframes={}
all_data = []  # List to store all individual dataframes

for file in csv_files:
    file_path = os.path.join(dir_path, file)
    df = pd.read_csv(file_path)
    key_name = os.path.splitext(file)[0]
    dataframes[key_name] = df
    all_data.append(df)

# Concatenate all individual dataframes to form a single dataframe
combined_df = pd.concat(all_data, ignore_index=True)

In [127]:
#
# combined_df

In [4]:
# remove dollar symbol
combined_df['Profit/Loss'] = combined_df['Profit/Loss'].str.replace('$','', regex = False)
combined_df['Trade Price'] = combined_df['Trade Price'].str.replace('$','', regex = False)
combined_df['Stock Price'] = combined_df['Stock Price'].str.replace('$','', regex = False)
combined_df['Adjusted Stock Price'] = combined_df['Adjusted Stock Price'].str.replace('$','', regex = False)

# convert in numbers
combined_df['Profit/Loss']= pd.to_numeric(combined_df['Profit/Loss'],errors='coerce')
combined_df['Trade Price']= pd.to_numeric(combined_df['Trade Price'],errors='coerce')

# convert date 
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df = combined_df.reset_index(drop=True)


In [129]:
#combined_df.to_excel('combined_df.xlsx')


### Filter stocks

In [5]:
# Add year column and Y_Symbol column
combined_df['Year'] = combined_df['Date'].dt.year
combined_df['Y_Symbol'] = combined_df['Year'].astype(str).str.cat(combined_df['Symbol'], sep='')

# select only stocks from file 'InputStocks.xlsx'
combined_df = combined_df[combined_df['Y_Symbol'].isin(df_stock['Y_Symbol'])]

# sort by date to build chart
combined_df = combined_df.sort_values(by='Date')

### Calculate call spread price

In [6]:
# calculate CS open price
combined_df.loc[combined_df["Size"] == -1, "Trade Price"] *= -1  # if size -1 then we sold options it's reduce our calls costs
grouped_by_date = combined_df.groupby(['Date','Symbol'])[['Trade Price', 'Profit/Loss']].sum().reset_index()


In [7]:
# sorf dataframe by symbol and date
sorted_df = grouped_by_date.sort_values(by=['Symbol','Date'])


In [8]:
# delete result values of option // clean df
sorted_df.loc[sorted_df['Profit/Loss']!=0, 'Trade Price']=0


In [133]:
#sorted_df.to_excel('sorted.xlsx')

### Call spread price filter

In [9]:
# Here I delete rows where price_limit exceed my limit


# Identify rows where "Trade Price" is greater than price_limit
to_drop = sorted_df[sorted_df['Trade Price'] > price_limit]

# For each of these rows, identify the next row with the same "Symbol" and a subsequent date
drop_indices = []
for idx, row in to_drop.iterrows():
    drop_indices.append(idx)
    next_row = sorted_df[(sorted_df['Symbol'] == row['Symbol']) & (sorted_df['Date'] > row['Date'])].head(1)
    if not next_row.empty:
        drop_indices.append(next_row.index[0])

# Drop these rows from the DataFrame
sorted_df = sorted_df.drop(drop_indices)


In [13]:
# sorf dataframe by date
sorted_df = sorted_df.sort_values(by='Date')

### Working with cummulative columns

In [14]:
# making cummulative sum of profit
df_chart = sorted_df.copy()
df_chart['CumSum'] = df_chart['Profit/Loss'].cumsum()

# add SPY price to compare

start_date = df_chart['Date'].min()
end_date = df_chart['Date'].max()

# Fetch SPY data for the given date range
spy_data = yf.download('SPY', start=start_date, end=end_date)

# Merge the data
df_chart = df_chart.merge(spy_data[['Close']], left_on='Date', right_index=True, how='left')
df_chart.rename(columns={'Close': 'SPY Price'}, inplace=True)

[*********************100%***********************]  1 of 1 completed


In [15]:
df_chart

,Date,Symbol,Trade Price,Profit/Loss,CumSum,SPY Price
0,2007-01-03,C,0.30,0.0,0.0,141.369995
1,2007-01-03,GE,0.20,0.0,0.0,141.369995
2,2007-01-08,BAC,0.40,0.0,0.0,141.190002
3,2007-01-09,MSFT,0.70,0.0,0.0,141.070007
4,2007-01-09,T,0.45,0.0,0.0,141.070007
...,...,...,...,...,...,...
1044,2021-10-20,JNJ,0.00,24.0,9959.0,452.410004
1047,2021-10-27,MSFT,0.00,238.0,10197.0,453.940002
1048,2021-10-27,V,0.00,-129.0,10068.0,453.940002
1049,2021-10-29,AAPL,0.00,113.0,10181.0,459.250000


In [16]:
df_chart.to_excel('df_chart.xlsx')

In [17]:
# Load the provided dataframe

# Plot using plotly.express
fig = px.line(df_chart, x='Date', y='CumSum', title='Cumulative Sum over Date')

#fig.add_trace(go.Scatter(x=df_chart_comp['Date'], y=df_chart_comp['SPY_buy_hold'], mode='lines', name='SPY_buy_hold'))

fig.show()



## Metrics

### Annualized return

In [35]:
# # calculations for SPY
# # Drop rows with NaN values in 'SPY Price' column
# spy_starting_price = df_chart_comp['SPY Price'].iloc[0]
# df_chart_comp_cleaned = df_chart_comp.dropna(subset=['SPY Price'])

# # Get the new ending price of SPY
# spy_ending_price_cleaned = df_chart_comp_cleaned['SPY Price'].iloc[-1]

# # Recalculate the total return for SPY
# spy_total_return_cleaned = (spy_ending_price_cleaned / spy_starting_price) - 1

# spy_total_return_cleaned


In [36]:
# ### Annualized return
# total_return = df_chart_comp['Profit/Loss'].sum()
# # Calculate the total number of days the strategy ran
# num_days = (df_chart_comp['Date'].iloc[-1] - df_chart_comp['Date'].iloc[0]).days

# # Calculate the annualized return using the total return and number of days
# annualized_return = ((1 + total_return / df_chart_comp['CumSum'].iloc[0]) ** (365.0 / num_days)) - 1

# annualized_return = round(annualized_return*100,2)
# annualized_return

In [37]:
# ### Annualized return SPY
# # Calculate the total number of days the data covers (for the cleaned data)
# num_days_spy = (df_chart_comp_cleaned['Date'].iloc[-1] - df_chart_comp_cleaned['Date'].iloc[0]).days

# # Calculate the annualized return for SPY using the total return and number of days
# spy_annualized_return = ((1 + spy_total_return_cleaned) ** (365.0 / num_days_spy)) - 1

# spy_annualized_return =round(spy_annualized_return*100,2)
# spy_annualized_return


### Volatility (Standard Deviation):

In [38]:
# # 1. Compute the daily returns
# df_chart_comp['Daily Returns'] = df_chart_comp['CumSum'].pct_change()

# # 2. Calculate the standard deviation of these daily returns
# daily_volatility = df_chart_comp['Daily Returns'].std()

# # 3. Annualize the standard deviation
# annualized_volatility = round(daily_volatility * (252**0.5),2)  # Using 252 trading days in a year

# annualized_volatility


### Drawdown

In [149]:
# # 1. Create a new column for cumulative balance
# df_chart_comp['Cumulative Balance'] = init_cap + df_chart_comp['Profit/Loss'].cumsum()

# # 2. Calculate running max for the cumulative balance
# running_max_balance = df_chart_comp['Cumulative Balance'].cummax()

# # 3. Calculate drawdowns as the decline from the running max balance
# drawdowns_balance = (df_chart_comp['Cumulative Balance'] - running_max_balance) / running_max_balance

# # 4. Identify the maximum drawdown
# max_drawdown_balance = drawdowns_balance.min()

# max_drawdown_balance


C:\Users\Vova\AppData\Local\Temp\ipykernel_13984\1803054491.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



-0.87201442091032

### consecutive loss trades
 

In [151]:
# 1. Create a binary column for loss days
df_loss_count = df_chart_comp.copy()
df_loss_count['Loss Day'] = (df_loss_count['Profit/Loss'] < 0).astype(int)

# 2. Calculate the cumulative sum of loss days, but reset to zero whenever a non-loss day is encountered
df_loss_count['Consecutive Loss Days'] = df_loss_count['Loss Day'].groupby((df_chart_comp['Loss Day'] == 0).cumsum()).cumsum()

# 3. Find the maximum number of consecutive loss days
max_consecutive_loss_days = df_loss_count['Consecutive Loss Days'].max()

max_consecutive_loss_days


7

In [41]:
#df_chart_comp_cleaned

## sharpe ratio

In [42]:
#average_annual = df_annual['Annual Profit'].mean()

## Sortino Ratio